In [1]:
import pandas as pd
import boto3
from botocore.config import Config
import os
from faker import Faker
from dotenv import load_dotenv
load_dotenv()
from datetime import datetime
import numpy as np

In [2]:
df=pd.read_excel(r'C:\Users\shush\Documents\GitHub\bhpmJS\tools\auctionInfo\Auction Final (1).xlsx')

In [ ]:
df

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   lot                208 non-null    int64 
 1   title              208 non-null    object
 2   startingPrice      208 non-null    int64 
 3   estimatedPriceMin  208 non-null    int64 
 4   estimatedPriceMax  208 non-null    int64 
 5   description        208 non-null    object
 6   titleEng           208 non-null    object
 7   descriptionEng     208 non-null    object
 8   Condition          37 non-null     object
 9   Provenance         6 non-null      object
dtypes: int64(4), object(6)
memory usage: 16.4+ KB


In [4]:
# Replace all Nan values to empty string
df = df.replace(np.nan, '', regex=True)

In [5]:
fileList=os.listdir(r'auctionInfo/')
fileList

['100_1.jpg',
 '100_2.jpg',
 '100_3.jpg',
 '100_4.jpg',
 '101_1.jpg',
 '101_2.jpg',
 '101_3.jpg',
 '101_4.jpg',
 '101_5.jpg',
 '102_1.jpg',
 '102_2.jpg',
 '102_3.jpg',
 '102_4.jpg',
 '102_5.jpg',
 '103_1.jpg',
 '103_2.jpg',
 '103_3.jpg',
 '103_4.jpg',
 '103_5.jpg',
 '104_1.jpg',
 '104_2.jpg',
 '104_3.jpg',
 '104_4.jpg',
 '104_5.jpg',
 '104_6.jpg',
 '104_7.jpg',
 '105_1.jpg',
 '105_2.jpg',
 '105_3.jpg',
 '105_4.jpg',
 '105_5.jpg',
 '106_1.jpg',
 '106_2.jpg',
 '106_3.jpg',
 '106_4.jpg',
 '107_1.jpg',
 '107_2.jpg',
 '107_3.jpg',
 '107_4.jpg',
 '107_5.jpg',
 '107_6.jpg',
 '107_7.jpg',
 '108_1.jpg',
 '108_2.jpg',
 '108_3.jpg',
 '109_1.jpg',
 '109_2.jpg',
 '109_3.jpg',
 '109_4.jpg',
 '109_5.jpg',
 '10_1.jpg',
 '10_2.jpg',
 '110_1.jpg',
 '110_2.jpg',
 '110_3.jpg',
 '110_4.jpg',
 '111_1.jpg',
 '111_2.jpg',
 '111_3.jpg',
 '111_4.jpg',
 '112_1.jpg',
 '112_2.jpg',
 '112_3.jpg',
 '112_4.jpg',
 '112_5.jpg',
 '113_1.jpg',
 '113_2.jpg',
 '113_3.jpg',
 '113_4.jpg',
 '113_5.jpg',
 '113_6.jpg',
 '114_1.

In [ ]:
for i in fileList:
    print(i.split('_')[0])

In [6]:
fake = Faker(['it_IT', 'en_US', 'ja_JP','zh_CN','zh_TW'])

In [7]:
auctionsID='fdfa3b15-1731-43fc-b23e-30c1707b954c'

In [8]:
my_config = Config(
    region_name = 'us-west-1',
    signature_version = 'v4',
    retries = {
        'max_attempts': 10,
        'mode': 'standard'
    }
)

In [9]:
client = boto3.client(
    'dynamodb',
    aws_access_key_id=os.getenv('awsAccessKeyId'),
    aws_secret_access_key=os.getenv('awsSecretAccessKey'),
    config=my_config)
dynamodb = boto3.resource(
    'dynamodb',
    aws_access_key_id=os.getenv('awsAccessKeyId'),
    aws_secret_access_key=os.getenv('awsSecretAccessKey'),
    config=my_config)
ddb_exceptions = client.exceptions

In [10]:
df.columns

Index(['lot', 'title', 'startingPrice', 'estimatedPriceMin',
       'estimatedPriceMax', 'description', 'titleEng', 'descriptionEng',
       'Condition', 'Provenance'],
      dtype='object')

In [ ]:
now = datetime.now()
now

In [ ]:
df.loc[5]['lot'].split(' ')[1]

In [11]:
auctionItemTable = dynamodb.Table('AuctionItem-o44se66l2zhuvdxs5ctmuxmmdy-dev')
lotsTable=dynamodb.Table('Lots-o44se66l2zhuvdxs5ctmuxmmdy-dev')

In [12]:
for i in range(df.shape[0]):
    # print(df.loc[i])
    createdAt=datetime.now().strftime("%Y-%m-%dT%H:%M:%S.000Z")
    auctionItemID=fake.uuid4()
    auctionItemData={'__typename': 'AuctionItem',
    'id':auctionItemID,
    'categoryID':'陶瓷',
    'description':df.loc[i]['description'],
    'name': df.loc[i]['title'],
    'title':df.loc[i]['title'],
    'titleEng':df.loc[i]['titleEng'],
    'descriptionEng':df.loc[i]['descriptionEng'],
    'condition': df.loc[i]['Condition'],
    'provenance': df.loc[i]['Provenance'],
    'createdAt':createdAt,
    'updatedAt':createdAt
    }
    #print(data)
    imgList=[]
    for pictureName in fileList:
        if str(df.loc[i]['lot']) == str(pictureName.split('_')[0]):
            print( df.loc[i]['lot'], pictureName.split('_')[0])
            print('有啦有啦')
            imgList.append(f"""https://bhpmjsaa65d4d2254e4b41a89df0d66c611dc0144251-prod.s3.us-west-1.amazonaws.com/public/auction1/{pictureName}""")
    auctionItemData['imgUrls']=imgList
    print(auctionItemData)
    print('上传auctionitem 开始')
    response = auctionItemTable.put_item(Item=auctionItemData)
    print('上传auctionitem 结束')

    lotsData={'__typename': 'Lots',
    'id':fake.uuid4(),
    'lot':int(df.loc[i]['lot']),
    'auctionItemID':auctionItemID,
    'auctionsID':auctionsID,
    'estimatedPriceMax':int(df.loc[i]['estimatedPriceMax']),
    'estimatedPriceMin':int(df.loc[i]['estimatedPriceMin']),
    'startingPrice':int(df.loc[i]['startingPrice']),
    'createdAt':createdAt,  
    'updatedAt':createdAt,  
    'lotsStatus':'ComingSoon'
    }
    
    print(lotsData)
    print("上传lotsData")
    response = lotsTable.put_item(Item=lotsData)
    print('上传lotsData 结束')


1 1
有啦有啦
1 1
有啦有啦
1 1
有啦有啦
1 1
有啦有啦
1 1
有啦有啦
{'__typename': 'AuctionItem', 'id': '929012f8-d623-4e37-81d4-b363b01565c5', 'categoryID': '陶瓷', 'description': '此壶为粉彩彩绘民国仕女图案，寿桃钮盖，壶身有字：梧桐清瑞，时壬辰年秋作，品相好，没有提梁，高8.5厘米，直径12厘米', 'name': '粉彩仕女提梁壶', 'title': '粉彩仕女提梁壶', 'titleEng': 'The Republic of China Period Teapot', 'descriptionEng': 'The body of the pot has a maid, and on the other side is the "wu tong qing rui" mark. H 8.5cm, D:12cm.', 'condition': '', 'provenance': '', 'createdAt': '2022-04-22T22:38:28.000Z', 'updatedAt': '2022-04-22T22:38:28.000Z', 'imgUrls': ['https://bhpmjsaa65d4d2254e4b41a89df0d66c611dc0144251-prod.s3.us-west-1.amazonaws.com/public/auction1/1_1.jpg', 'https://bhpmjsaa65d4d2254e4b41a89df0d66c611dc0144251-prod.s3.us-west-1.amazonaws.com/public/auction1/1_2.jpg', 'https://bhpmjsaa65d4d2254e4b41a89df0d66c611dc0144251-prod.s3.us-west-1.amazonaws.com/public/auction1/1_3.jpg', 'https://bhpmjsaa65d4d2254e4b41a89df0d66c611dc0144251-prod.s3.us-west-1.amazonaws.com/public/auction1/1